In [4]:
from gbd_mapping import risk_factors
from neonatal.components import LBWSGRisk

lbwsg = risk_factors.low_birth_weight_and_short_gestation
cats = LBWSGRisk.parse_lbwsg_categories(lbwsg.categories.to_dict())
cats.reset_index().sort_values(['gestation_time', 'birth_weight']).set_index(['gestation_time', 'birth_weight'])

cat
gestation_time birth_weight        
[0, 24)        [0, 500)        cat2
               [500, 1000)     cat8
[24, 26)       [500, 1000)    cat10
[26, 28)       [500, 1000)    cat11
               [1000, 1500)   cat17
[28, 30)       [500, 1000)    cat15
               [1000, 1500)   cat21
               [1500, 2000)   cat20
               [2000, 2500)   cat80
               [2500, 3000)   cat81
               [3000, 3500)   cat82
[30, 32)       [500, 1000)    cat14
               [1000, 1500)   cat23
               [1500, 2000)   cat27
               [2000, 2500)   cat26
               [2500, 3000)   cat88
               [3000, 3500)   cat89
               [3500, 4000)   cat90
[32, 34)       [1000, 1500)   cat22
               [1500, 2000)   cat29
               [2000, 2500)   cat30
               [2500, 3000)   cat33
               [3000, 3500)   cat95
               [3500, 4000)   cat96
[34, 36)       [1000, 1500)   cat19
               [1500, 2000)   cat28
               [2000, 2500)   cat34
               [2500, 3000)   cat37
               [3000, 3500)   cat39
               [3500, 4000)   cat41
               [4000, 4500)   cat38
[36, 37)       [1000, 1500)  cat106
               [1500, 2000)   cat25
               [2000, 2500)   cat36
               [2500, 3000)   cat40
               [3000, 3500)   cat45
               [3500, 4000)   cat47
               [4000, 4500)   cat46
[37, 38)       [1500, 2000)   cat24
               [2000, 2500)   cat35
               [2500, 3000)   cat42
               [3000, 3500)   cat48
               [3500, 4000)   cat50
               [4000, 4500)   cat49
[38, 40)       [1000, 1500)  cat116
               [1500, 2000)  cat117
               [2000, 2500)   cat32
               [2500, 3000)   cat44
               [3000, 3500)   cat52
               [3500, 4000)   cat54
               [4000, 4500)   cat53
[40, 42)       [1500, 2000)  cat123
               [2000, 2500)   cat31
               [2500, 3000)   cat43
               [3000, 3500)   cat51
               [3500, 4000)   cat55
               [4000, 4500)   cat56

In [33]:
import pandas as pd
def collapse_gt(x):
    return pd.Interval(x.gestation_time.min().left, x.gestation_time.max().right, closed='left')

lbwsg = risk_factors.low_birth_weight_and_short_gestation
cats = LBWSGRisk.parse_lbwsg_categories(lbwsg.categories.to_dict())
cats = cats.reset_index().groupby('birth_weight').apply(collapse_gt).rename('gt_range')
cats

birth_weight
[0, 500)         [0, 24)
[500, 1000)      [0, 32)
[1000, 1500)    [26, 40)
[1500, 2000)    [28, 42)
[2000, 2500)    [28, 42)
[2500, 3000)    [28, 42)
[3000, 3500)    [28, 42)
[3500, 4000)    [30, 42)
[4000, 4500)    [34, 42)
Name: gt_range, dtype: object

In [57]:
MAX_BW = cats.index.max().right
MAX_GT = cats.max().right

SHIFT_IN_RANGE = 0.001

def nn_shift(bw, gt, bw_shift, gt_shift):
    bw_s = bw + bw_shift
    gt_s = gt + gt_shift
    if bw_s >= MAX_BW:
        bw_s = MAX_BW - SHIFT_WITHIN_RANGE  # BUG here - if we shift off the edge to the left, 
                                            # we should be using table 2 from notes to look up based on gt
    if gt_s >= MAX_GT:
        gt_s = MAX_GT - SHIFT_WITHIN_RANGE
    
    idx = cats.index.get_loc(bw_s)
    bw_range = cats.index[idx]
    gt_range = cats.iloc[idx]
    print(bw_range)
    print(gt_range)
    if gt_s < gt_range.left:    # A
        bw_s = bw_range.left - SHIFT_IN_RANGE
    if gt_s >= gt_range.right:  # B
        gt_s = gt_range.right - SHIFT_IN_RANGE
    
    return (bw_s, gt_s)

In [50]:
# case A - diagonal
nn_shift(550, 23, 500, 2)

(999.999, 25)

In [53]:
# case A - vertical
nn_shift(1250, 38, 0, 3)

(1250, 39.999)

In [51]:
# case B - diagonal
nn_shift(750, 31, 100, 2)

(850, 31.999)

In [54]:
# case B - horizontal
nn_shift(3500, 30, 1000, 0)

(3999.999, 30)

In [52]:
# case C - diagonal
nn_shift(4200, 41.25, 400, 2)

(4499.999, 41.999)

In [55]:
# off the edge vertically
nn_shift(2020, 41, 0, 5)

(2020, 41.999)

In [58]:
# off the edge horizontally - BUG!!!!!
nn_shift(999, 23, 4000, 0)

[4000, 4500)
[34, 42)


(3999.999, 23)